In [30]:
import sys, os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import datasets

from PIL import Image
from tqdm import tqdm
from annoy import AnnoyIndex


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

main_repo = '/home/baba/Bureau/ProjetPoster/'
dev_repo = os.path.join(os.getcwd(), "dev")
sys.path.append(dev_repo)

from loader import *

In [31]:
#generate data
df_data = pd.read_csv(main_repo + "movies_metadata.csv")

print("Overview NaN :", (df_data.overview.isna().sum()))
df_data = df_data.dropna(subset="overview")
df_data = df_data[["overview", "title"]]

#split data
df_train, df_test = train_test_split(df_data, test_size=0.5, random_state=42)

Overview NaN : 954


/tmp/ipykernel_5255/2877568664.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(main_repo + "movies_metadata.csv")


## Bag of Words

In [32]:
vectorizer = CountVectorizer(stop_words='english')
train_count_matrix = vectorizer.fit_transform(df_train.overview)

len(vectorizer.vocabulary_)

53363

In [33]:
train_count_matrix.shape

(22256, 53363)

## Annoy

In [34]:
#annoy
annoy_index = AnnoyIndex(len(vectorizer.vocabulary_), 'angular')

for i in tqdm(range(train_count_matrix.shape[0])):
    annoy_index.add_item(i, train_count_matrix[i].toarray().flatten())

annoy_index.build(10)

100%|██████████| 22256/22256 [00:57<00:00, 388.17it/s]


True

In [53]:
def search(query_vector, k=5):
    indices = annoy_index.get_nns_by_vector(query_vector, k)
    titles = df_train['title'].iloc[indices]  
    return titles

index = 10
query_vector = train_count_matrix.toarray()[index]
result = search(query_vector)

In [54]:
df_train.iloc[index]["title"]

'The Aristocats'

In [55]:
result

608      The Aristocats
27572     The Black Cat
26174      Swedenhielms
19651           Transit
40065     You Lucky Dog
Name: title, dtype: object

## NEURAL NETWORK

## Annoy